In [1]:
import sys

sys.path.append('../')

In [10]:
import boto3
import sagemaker
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from src.mnist_sagemaker_ci_cd.lib.settings import Settings


## Sagemaker Endpoint Deployment
In this notebook, we describe the steps to invoke an endpoint once its been deployed via Sagemaker. The key thing to learn here is that the `sagemaker.predictor.Predictor` class is used to invoke the endpoint. This can be effectively wrapped around a FastAPI app to create a REST API since that's not really exposed by Sagemaker.

In [11]:
SESSION = sagemaker.Session(boto3.Session(region_name="us-east-1"))
IAM_ROLE = "arn:aws:iam::220582896887:role/programmatic-aws-sagemaker-role-access"
SETTINGS = Settings()

In [12]:
predictor = Predictor(
    endpoint_name = SETTINGS.github_sha[:7],
    sagemaker_session = SESSION,
    serializer = IdentitySerializer(),
    deserializer = JSONDeserializer()
)

In [13]:
with open('../data/4.png', 'rb') as f:
    payload = f.read()

In [14]:
predictor.predict(payload)

{'prediction': [6]}

## Deleting the endpoint. 

In the event that the endpoint doesn't satisfy the constraints of the business, it can be deleted using the `delete_endpoint` method.

```python
predictor.delete_endpoint()
```

### Running this in a Github Action

```yaml
  close_job:
    # this job will only run if the PR has been closed without being merged
    if: github.event.pull_request.merged == false
    runs-on: ubuntu-latest
    steps:
    - run: |
        echo PR #${{ github.event.number }} has been closed without being merged
```


## Invoking Endpoint using only `boto3`
Lamdba functions are notorious for being unable to accomodate multiple python packages. To reduce the burden of the number of packages, we can use `boto3` to invoke the endpoint. 

In [3]:
import sys

sys.path.append('../')

In [4]:
import json

import boto3
from src.mnist_sagemaker_ci_cd.lib.settings import Settings

SETTINGS = Settings()
sagemaker_runtime = boto3.client('sagemaker-runtime')
with open('../data/4.png', 'rb') as f:
    payload = f.read()
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=SETTINGS.github_sha[:7],
            ContentType='application/octet-stream',  # Change this depending on your payload format
            Accept='application/json',
            Body=payload
        )
    except Exception as e:
        raise(e)
json_body = json.loads(response['Body'].read().decode('utf-8'))
json_body['prediction'][0]

6